In [ ]:
import re
import os
import io
import time
import unicodedata

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from sklearn.model_selection import train_test_split

In [ ]:
# Download the file

path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin = 'http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract = True)

path_to_file = os.path.dirname(path_to_zip) + "/spa-eng/spa.txt"